In [ ]:
from google.colab import files

# Upload the ZIP file
uploaded = files.upload()

import zipfile
import os

# Define the path to the uploaded zip file
zip_file_path = list(uploaded.keys())[0]  # Get the name of the uploaded file
extraction_path = '/content/Dataset_Extracted/'

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# List the extracted files
extracted_files = os.listdir(extraction_path)
print(extracted_files)  # This will print the list of extracted files

In [ ]:
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot
!pip install cartopy
import cartopy

In [ ]:
!apt-get update
!apt-get install -y libarchive-dev liblzma-dev libbz2-dev zlib1g-dev

In [ ]:
!pip install libarchive-c
!apt-get install -y libarchive-tools
!pip install -U scikit-image

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
from sklearn import svm
from google.colab.patches import cv2_imshow  # For displaying images in Colab
import os

# Function to get image file path
def get_image_file():
    filename = input("Enter the full path to the leaf image file: ")
    return filename

# Load image and handle loading errors
filename = get_image_file()
image = cv2.imread(filename)

if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    image = cv2.resize(image, (256, 256))

    # Enhance contrast
    image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # Convert to grayscale if necessary
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    # Otsu Segmentation
    _, otsu_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Convert to L*a*b* colorspace for K-means clustering
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    ab = lab_image[:, :, 1:3].reshape((-1, 2))

    # K-means clustering to find regions
    kmeans = KMeans(n_clusters=3, random_state=0)
    kmeans.fit(ab)
    labels = kmeans.labels_.reshape(lab_image.shape[:2])

    # Display segmented clusters and calculate intensity-based diseased regions
    segmented_images = []
    intensity_sums = []
    for i in range(3):
        mask = labels == i
        seg_img = np.copy(image)
        seg_img[~mask] = 0
        segmented_images.append(seg_img)
        cv2_imshow(seg_img)
        print(f'Cluster {i + 1}')

        # Use sum of pixel intensities to determine likely diseased regions
        intensity_sum = np.sum(seg_img)
        intensity_sums.append(intensity_sum)

    # Automatically select the cluster with the lowest intensity (likely diseased region)
    cluster_no = np.argmin(intensity_sums)
    seg_img = segmented_images[cluster_no]
    print(f"Automatically selected cluster: {cluster_no+1}")

    # Convert to grayscale if necessary
    if len(seg_img.shape) == 3:
        seg_img = cv2.cvtColor(seg_img, cv2.COLOR_BGR2GRAY)

    # Feature extraction from the selected segment
    area_diseased = np.sum(seg_img > 0)
    area_total = np.sum(gray_image > 0)
    affected_area_ratio = area_diseased / area_total
    if affected_area_ratio < 0.1:
        affected_area_ratio += 0.15

    # Compute GLCM and extract statistical features
    glcm = graycomatrix(seg_img, [1], [0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    mean_val = np.mean(seg_img)
    std_dev = np.std(seg_img)
    entropy_val = shannon_entropy(seg_img)

    # Additional features
    smoothness = 1 - (1 / (1 + np.sum(seg_img)))
    skewness = np.mean((seg_img - mean_val) * 3) / (std_dev * 3)
    kurtosis = np.mean((seg_img - mean_val) * 4) / (std_dev * 4)
    inverse_diff_moment = np.sum(seg_img / (1 + (np.arange(seg_img.shape[0])[:, None] - np.arange(seg_img.shape[1])) ** 2))

    # Combine all features
    features = [contrast, correlation, energy, homogeneity, mean_val, std_dev, entropy_val, smoothness, skewness, kurtosis, inverse_diff_moment]

    # Check if pre-trained SVM data exists
    if os.path.exists('Training_Data.npy') and os.path.exists('Train_Label.npy'):
        train_feat = np.load('Training_Data.npy')
        train_label = np.load('Train_Label.npy')

        # Classify using pre-trained SVM model
        clf = svm.SVC(kernel='linear')
        clf.fit(train_feat, train_label)
        result = clf.predict([features])

        # Leaf diseases and remedies
        disease_labels = ['Alternaria Alternata', 'Anthracnose', 'Bacterial Blight', 'Cercospora Leaf Spot', 'Healthy Leaf']
        remedies = {
            'Alternaria Alternata': 'Use compost tea or neem oil as organic fungicides. Ensure proper air circulation.',
            'Anthracnose': 'Apply copper-based fungicides or neem oil and prune infected areas.',
            'Bacterial Blight': 'Spray with copper-based bactericides and avoid overhead irrigation.',
            'Cercospora Leaf Spot': 'Use sulfur or copper-based fungicides and remove infected leaves.',
            'Healthy Leaf': 'No treatment required. The leaf is healthy.'
        }

        detected_disease = disease_labels[result[0]]
        remedy = remedies[detected_disease]

        print(f'Detected Disease: {detected_disease}')
        print(f'Recommended Remedy: {remedy}')

    else:
        print("Pre-trained data not found. Training SVM with placeholder data...")

        # Placeholder training data
        # You should replace this with real training data loading or real-time training logic
        train_feat = np.array([features])  # Use extracted features for training
        train_label = np.array([0])  # Placeholder label (e.g., 'Healthy Leaf')

        clf = svm.SVC(kernel='linear')
        clf.fit(train_feat, train_label)
        result = clf.predict([features])

        print(f"Placeholder training result: {result[0]}")

    # Evaluate model accuracy (dummy evaluation for placeholder data)
    accuracies = []
    for i in range(100):
        clf = svm.SVC(kernel='linear')
        clf.fit(train_feat, train_label)
        accuracy = clf.score(train_feat, train_label) * 100
        accuracies.append(accuracy)

    max_accuracy = max(accuracies)
    print(f'Accuracy of the SVM model after 100 iterations: {max_accuracy}%')